In [129]:
%matplotlib inline
import pandas as pd
import geopandas as gpd
import numpy as np
import folium
from shapely.geometry import Polygon, Point 
import folium.colormap as cm

In [130]:
#global variable dictionary
gvd = {'lat_min': 0.0, 'lon_min': 0.0,
       'lat_step': 0.0, 'lon_step': 0.0}

In [124]:
def select_locs(df, max_lat = 42.017888, min_lat = 41.646487,
                    max_lon = -87.525492, min_lon = -87.821101):
    res = df[((df['Latitude'] < max_lat) & (df['Latitude'] > min_lat)) &
             ((df['Longitude'] < max_lon) & (df['Longitude'] > min_lon))].reset_index()
    return res.drop('index', axis=1, inplace=False, errors='raise')

def bin_locs(df, lat_segs, lon_segs):
    """lat_segs: number of segments (bins) in the notrth-south direction
       lon_segs: number of segments (bins) in the east-west direction"""
    lat_min = df['Latitude'].min()
    lon_min = df['Longitude'].min()

    lat_step = (df['Latitude'].max() - lat_min)/lat_segs
    lon_step = (df['Longitude'].max() - lon_min)/lon_segs
    
    res = pd.DataFrame(df)#.drop('index', axis=1, inplace=False, errors='raise')
    res['bin'] = None
    #bins are assigned coordinates as in maths
        #first coordinate increases from left to right (west to east)
        #second coordinate increases from bottom to top (south to north)
    for i in res.index:
        res.set_value(i, 'bin', (int(np.floor((res['Longitude'][i] - lon_min)/lon_step)),
                                 int(np.floor((res['Latitude'][i]  - lat_min)/lat_step))))
    return (lat_min, lon_min, lat_step, lon_step, res)

def getCornerLatLonForBin(bin_p):
    return (
        lat_min + (bin_p[1]*lat_step), 
        lon_min + (bin_p[0]*lon_step)
    )
    
def getPolygonForBinReverse(bin_p):
    lat, lon = getCornerLatLonForBin(bin_p)
    return Polygon([
            (lon,            lat),
            (lon + lon_step, lat),
            (lon + lon_step, lat + lat_step),
            (lon,            lat + lat_step),
        ])

def group(df):
    return df[['bin']].groupby(['bin']).size().reset_index().rename(columns={0:'count'})

def getGeoDataFrame(df, color_map):
    gdf = gpd.GeoDataFrame(df)
    gdf['geometry'] = gdf['bin'].map(lambda x: getPolygonForBinReverse(x))
    color_map.scale(gdf['count'].min(), gdf['count'].max())
    gdf['style'] = gdf['relative_count'].map(lambda x: {'fillColor' : linear(x), 'weight' : 0})
    return gdf



In [21]:
crimes = pd.read_pickle('crime_chicago_with_timestamp.pkl')

In this part we filter the crimes by their locations.

In [122]:
crimes_filtered = select_locs(crimes, max_lat = 41.911151, min_lat = 41.867355,
                                    max_lon = -87.623653, min_lon = -87.653335)
crimes_filtered

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,...,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location,day,datetime
0,10361478,HY553227,12/28/2015 11:08:00 PM,001XX W ELM ST,033A,ROBBERY,ATTEMPT: ARMED-HANDGUN,SIDEWALK,False,False,...,03,51682,93477,2015,01/04/2016 04:02:14 PM,41.903128,-87.632069,"(41.903127946, -87.632068763)",12/28/2015,2015-12-28
1,10361633,HY553192,12/28/2015 10:40:00 PM,006XX N KINGSBURY ST,0910,MOTOR VEHICLE THEFT,AUTOMOBILE,DRIVEWAY - RESIDENTIAL,False,False,...,07,49180,89898,2015,01/04/2016 04:02:14 PM,41.893363,-87.641365,"(41.893362725, -87.641365059)",12/28/2015,2015-12-28
2,10361917,HY553584,12/28/2015 08:00:00 PM,002XX S CANAL ST,0870,THEFT,POCKET-PICKING,OTHER RAILROAD PROP / TRAIN DEPOT,False,False,...,06,49707,84643,2015,01/04/2016 04:02:14 PM,41.878931,-87.639586,"(41.878931004, -87.639585621)",12/28/2015,2015-12-28
3,10362231,HY554024,12/28/2015 08:00:00 PM,008XX N LA SALLE DR,0810,THEFT,OVER $500,APARTMENT,False,False,...,06,51524,91247,2015,01/04/2016 04:02:14 PM,41.897012,-87.632716,"(41.897012256, -87.632716018)",12/28/2015,2015-12-28
4,10361457,HY553186,12/28/2015 07:45:00 PM,001XX S STATE ST,0860,THEFT,RETAIL THEFT,DEPARTMENT STORE,False,False,...,06,52931,85211,2015,01/04/2016 04:02:14 PM,41.880418,-87.627731,"(41.880417506, -87.627730673)",12/28/2015,2015-12-28
5,10361363,HY553087,12/28/2015 07:20:00 PM,001XX N STATE ST,0860,THEFT,RETAIL THEFT,DEPARTMENT STORE,False,False,...,06,52882,86334,2015,01/04/2016 04:02:14 PM,41.883500,-87.627877,"(41.883500187, -87.627876698)",12/28/2015,2015-12-28
6,10361390,HY553054,12/28/2015 07:14:00 PM,0000X E JACKSON BLVD,0486,BATTERY,DOMESTIC BATTERY SIMPLE,RESIDENCE,True,True,...,08B,53200,84416,2015,01/04/2016 04:02:14 PM,41.878230,-87.626767,"(41.878229908, -87.626766976)",12/28/2015,2015-12-28
7,10361483,HY553258,12/28/2015 07:00:00 PM,007XX N WABASH AVE,1320,CRIMINAL DAMAGE,TO VEHICLE,PARKING LOT/GARAGE(NON.RESID.),False,False,...,14,53128,90923,2015,01/04/2016 04:02:14 PM,41.896087,-87.626835,"(41.89608709, -87.626834633)",12/28/2015,2015-12-28
8,10361314,HY552995,12/28/2015 06:15:00 PM,001XX W LAKE ST,0460,BATTERY,SIMPLE,CTA PLATFORM,False,False,...,08B,51864,87142,2015,01/04/2016 04:02:14 PM,41.885740,-87.631591,"(41.885740288, -87.631590568)",12/28/2015,2015-12-28
9,10361336,HY552974,12/28/2015 06:00:00 PM,009XX N MICHIGAN AVE,0860,THEFT,RETAIL THEFT,DEPARTMENT STORE,False,False,...,06,53854,92140,2015,01/04/2016 04:02:14 PM,41.899410,-87.624131,"(41.899410159, -87.624131266)",12/28/2015,2015-12-28


In [127]:
lat_min, lon_min, lat_step, lon_step, crimes_binned = bin_locs(crimes_filtered, 40, 40)
crimes_binned

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,...,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location,day,datetime,bin
0,10361478,HY553227,12/28/2015 11:08:00 PM,001XX W ELM ST,033A,ROBBERY,ATTEMPT: ARMED-HANDGUN,SIDEWALK,False,False,...,51682,93477,2015,01/04/2016 04:02:14 PM,41.903128,-87.632069,"(41.903127946, -87.632068763)",12/28/2015,2015-12-28,"(28, 32)"
1,10361633,HY553192,12/28/2015 10:40:00 PM,006XX N KINGSBURY ST,0910,MOTOR VEHICLE THEFT,AUTOMOBILE,DRIVEWAY - RESIDENTIAL,False,False,...,49180,89898,2015,01/04/2016 04:02:14 PM,41.893363,-87.641365,"(41.893362725, -87.641365059)",12/28/2015,2015-12-28,"(16, 23)"
2,10361917,HY553584,12/28/2015 08:00:00 PM,002XX S CANAL ST,0870,THEFT,POCKET-PICKING,OTHER RAILROAD PROP / TRAIN DEPOT,False,False,...,49707,84643,2015,01/04/2016 04:02:14 PM,41.878931,-87.639586,"(41.878931004, -87.639585621)",12/28/2015,2015-12-28,"(18, 10)"
3,10362231,HY554024,12/28/2015 08:00:00 PM,008XX N LA SALLE DR,0810,THEFT,OVER $500,APARTMENT,False,False,...,51524,91247,2015,01/04/2016 04:02:14 PM,41.897012,-87.632716,"(41.897012256, -87.632716018)",12/28/2015,2015-12-28,"(27, 27)"
4,10361457,HY553186,12/28/2015 07:45:00 PM,001XX S STATE ST,0860,THEFT,RETAIL THEFT,DEPARTMENT STORE,False,False,...,52931,85211,2015,01/04/2016 04:02:14 PM,41.880418,-87.627731,"(41.880417506, -87.627730673)",12/28/2015,2015-12-28,"(34, 11)"
5,10361363,HY553087,12/28/2015 07:20:00 PM,001XX N STATE ST,0860,THEFT,RETAIL THEFT,DEPARTMENT STORE,False,False,...,52882,86334,2015,01/04/2016 04:02:14 PM,41.883500,-87.627877,"(41.883500187, -87.627876698)",12/28/2015,2015-12-28,"(34, 14)"
6,10361390,HY553054,12/28/2015 07:14:00 PM,0000X E JACKSON BLVD,0486,BATTERY,DOMESTIC BATTERY SIMPLE,RESIDENCE,True,True,...,53200,84416,2015,01/04/2016 04:02:14 PM,41.878230,-87.626767,"(41.878229908, -87.626766976)",12/28/2015,2015-12-28,"(35, 9)"
7,10361483,HY553258,12/28/2015 07:00:00 PM,007XX N WABASH AVE,1320,CRIMINAL DAMAGE,TO VEHICLE,PARKING LOT/GARAGE(NON.RESID.),False,False,...,53128,90923,2015,01/04/2016 04:02:14 PM,41.896087,-87.626835,"(41.89608709, -87.626834633)",12/28/2015,2015-12-28,"(35, 26)"
8,10361314,HY552995,12/28/2015 06:15:00 PM,001XX W LAKE ST,0460,BATTERY,SIMPLE,CTA PLATFORM,False,False,...,51864,87142,2015,01/04/2016 04:02:14 PM,41.885740,-87.631591,"(41.885740288, -87.631590568)",12/28/2015,2015-12-28,"(29, 16)"
9,10361336,HY552974,12/28/2015 06:00:00 PM,009XX N MICHIGAN AVE,0860,THEFT,RETAIL THEFT,DEPARTMENT STORE,False,False,...,53854,92140,2015,01/04/2016 04:02:14 PM,41.899410,-87.624131,"(41.899410159, -87.624131266)",12/28/2015,2015-12-28,"(39, 29)"


In [128]:
getGeoDataFrame(group(crimes_binned))

,bin,count,geometry
0,"(0, 1)",63,"POLYGON ((-87.65333436 41.868451384475, -87.65..."
1,"(0, 2)",78,"POLYGON ((-87.65333436 41.86954623795, -87.652..."
2,"(0, 3)",57,"POLYGON ((-87.65333436 41.870641091425, -87.65..."
3,"(0, 4)",55,"POLYGON ((-87.65333436 41.8717359449, -87.6525..."
4,"(0, 5)",3,"POLYGON ((-87.65333436 41.872830798375, -87.65..."
5,"(0, 6)",8,"POLYGON ((-87.65333436 41.87392565185, -87.652..."
6,"(0, 7)",11,"POLYGON ((-87.65333436 41.875020505325, -87.65..."
7,"(0, 8)",63,"POLYGON ((-87.65333436 41.8761153588, -87.6525..."
8,"(0, 9)",64,"POLYGON ((-87.65333436 41.877210212275, -87.65..."
9,"(0, 10)",41,"POLYGON ((-87.65333436 41.87830506575, -87.652..."
